<a href="https://colab.research.google.com/github/DaeSeokSong/MachineLearningModels/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Drive Local Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install Morpheme analyzer
*   [Reference](https://soohee410.github.io/compare_tagger)
*   [Install](https://sanghyu.tistory.com/170)

In [ ]:
# install okt, komoran kkma
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

# set env
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

# install mecab
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

# Import
*   [KoNLPy Reperence](https://konlpy-ko.readthedocs.io/ko/v0.4.3/)



In [ ]:
# import for MechineLearning
import tensorflow as tf
import pandas as pd
import numpy as np

# import Morpheme analyzer
from konlpy.tag import Kkma, Komoran, Okt, Mecab
from konlpy.utils import pprint

#Main

In [ ]:
mecab = Mecab()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

text = "안녕하세요 형태소 분석 테스트 중입니다."

print("##### mecab #####")
pprint(mecab.morphs(text))
print("##### kkma #####")
pprint(kkma.pos(text))
print("##### komoran #####")
pprint(komoran.pos(text))
print("##### okt #####")
pprint(okt.pos(text))